#### In this example we will look at how to use the K-Nearest_Neighbor algorithm for classification. We will use a modified version of the Video Store data set for this example. We will use the "Incidentals" attribute as the target attribute for classification (the class attribute). The goal is to be able to classify an unseen instance as "Yes" or "No" given the values of "Incidentals" from training instances.

In [1]:
import numpy as np
import pandas as pd

In [2]:
cd C:\Users\mobasher\Documents\Data

[Errno 2] No such file or directory: 'C:UsersmobasherDocumentsData'
/Users/appobs/Desktop/hw/478/week3


In [3]:
vstable = pd.read_csv("/Users/appobs/Desktop/hw/478/week3/Video_Store_2.csv", index_col=0)
vstable.shape

(50, 7)

In [4]:
vstable.head()

,Gender,Income,Age,Rentals,Avg Per Visit,Genre,Incidentals
Cust ID,,,,,,,
1,M,45000,25,32,2.5,Action,Yes
2,F,54000,33,12,3.4,Drama,No
3,F,32000,20,42,1.6,Comedy,No
4,F,59000,70,16,4.2,Drama,Yes
5,M,37000,35,25,3.2,Action,Yes


#### We will be splitting the data into a test and training partions with the test partition to be used for evaluating model error-rate and the training partition to be used to find the K nearest neighbors. Before spliting the data we need to do a random reshuffling to make sure the instances are randomized.

In [5]:
vs = vstable.reindex(np.random.permutation(vstable.index))
vs.head(10)

,Gender,Income,Age,Rentals,Avg Per Visit,Genre,Incidentals
Cust ID,,,,,,,
1,M,45000,25,32,2.5,Action,Yes
42,M,32000,25,26,2.2,Action,Yes
23,F,2000,15,30,2.5,Comedy,No
31,F,49000,56,15,3.2,Comedy,No
16,M,17000,19,26,2.2,Action,Yes
2,F,54000,33,12,3.4,Drama,No
32,M,47000,30,21,3.1,Drama,Yes
18,F,6000,16,39,1.8,Action,Yes
27,F,62000,47,32,3.6,Drama,No


In [6]:
len(vs)

50

In [7]:
vs_names = vs.columns.values
vs_names

array(['Gender', 'Income', 'Age', 'Rentals', 'Avg Per Visit', 'Genre',
       'Incidentals'], dtype=object)

#### The target attribute for classification is Incidentals:

In [8]:
vs_target = vs.Incidentals

#### Before we can compute distances we need to convert the data (excluding the target attribute containing the class labels) into standard spreadsheet format with binary dummy variables for categorical attributes).

In [9]:
vs = pd.get_dummies(vs[['Gender','Income','Age','Rentals','Avg Per Visit','Genre']])
vs.head(10)
vs_names = vs.columns.values
vs_names

array(['Income', 'Age', 'Rentals', 'Avg Per Visit', 'Gender_F',
       'Gender_M', 'Genre_Action', 'Genre_Comedy', 'Genre_Drama'],
      dtype=object)

#### To be able to evaluate the accuracy of our predictions, we will split the data into training and test sets. In this case, we will use 80% for training and the remaining 20% for testing. Note that we must also do the same split to the target attribute.

In [10]:
tpercent = 0.8
tsize = int(np.floor(tpercent * len(vs)))
vs_train = vs[:tsize]
vs_test = vs[tsize:]

In [11]:
print (vs_train.shape)
print (vs_test.shape)

(40, 9)
(10, 9)


In [12]:
np.set_printoptions(suppress=True, linewidth=120)

vs_train.head(10)

,Income,Age,Rentals,Avg Per Visit,Gender_F,Gender_M,Genre_Action,Genre_Comedy,Genre_Drama
Cust ID,,,,,,,,,
1,45000,25,32,2.5,0,1,1,0,0
42,32000,25,26,2.2,0,1,1,0,0
23,2000,15,30,2.5,1,0,0,1,0
31,49000,56,15,3.2,1,0,0,1,0
16,17000,19,26,2.2,0,1,1,0,0
2,54000,33,12,3.4,1,0,0,0,1
32,47000,30,21,3.1,0,1,0,0,1
18,6000,16,39,1.8,1,0,1,0,0
27,62000,47,32,3.6,1,0,0,0,1


In [13]:
vs_test

,Income,Age,Rentals,Avg Per Visit,Gender_F,Gender_M,Genre_Action,Genre_Comedy,Genre_Drama
Cust ID,,,,,,,,,
29,15000,18,37,2.1,1,0,1,0,0
46,57000,43,9,1.1,1,0,0,0,1
19,24000,25,41,3.1,1,0,0,1,0
48,52000,47,14,1.6,1,0,0,0,1
9,38000,21,18,2.1,0,1,0,1,0
33,23000,25,28,2.7,0,1,1,0,0
8,74000,25,31,2.4,0,1,1,0,0
25,1000,16,25,1.4,0,1,0,1,0
6,18000,20,29,1.7,0,1,1,0,0


#### Splitting the target attribute ("Incidentals") accordingly:

In [14]:
vs_target_train = vs_target[0:int(tsize)]
vs_target_test = vs_target[int(tsize):len(vs)]

In [15]:
vs_target_train.head()

Cust ID
1     Yes
42    Yes
23     No
31     No
16    Yes
Name: Incidentals, dtype: object

In [16]:
vs_target_test

Cust ID
29    Yes
46     No
19     No
48     No
9      No
33     No
8     Yes
25    Yes
6      No
20    Yes
Name: Incidentals, dtype: object

#### Next, we normalize the attributes so that everything is in [0,1] scale. We can use the normalization functions from the kNN module in Ch. 2 of the text. In this case, however, we will use the more flexible and robust scaler function from the preprocessing module of scikit-learn package.

In [17]:
from sklearn import preprocessing

In [18]:
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(vs_train)

/Users/appobs/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


MinMaxScaler(copy=True, feature_range=(0, 1))

In [19]:
vs_train_norm = min_max_scaler.fit_transform(vs_train)
vs_test_norm = min_max_scaler.fit_transform(vs_test)

/Users/appobs/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/appobs/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


#### Note that while these Scikit-learn functions accept Pandas dataframes as input, they return Numpy arrays (in this case the normalized versions of vs_train and vs_test).

In [20]:
np.set_printoptions(precision=2, linewidth=100)
vs_train_norm[:10]


array([[0.49, 0.18, 0.57, 0.37, 0.  , 1.  , 1.  , 0.  , 0.  ],
       [0.34, 0.18, 0.41, 0.29, 0.  , 1.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.51, 0.37, 1.  , 0.  , 0.  , 1.  , 0.  ],
       [0.54, 0.75, 0.11, 0.57, 1.  , 0.  , 0.  , 1.  , 0.  ],
       [0.17, 0.07, 0.41, 0.29, 0.  , 1.  , 1.  , 0.  , 0.  ],
       [0.6 , 0.33, 0.03, 0.63, 1.  , 0.  , 0.  , 0.  , 1.  ],
       [0.52, 0.27, 0.27, 0.54, 0.  , 1.  , 0.  , 0.  , 1.  ],
       [0.05, 0.02, 0.76, 0.17, 1.  , 0.  , 1.  , 0.  , 0.  ],
       [0.69, 0.58, 0.57, 0.69, 1.  , 0.  , 0.  , 0.  , 1.  ],
       [0.45, 0.42, 0.24, 0.6 , 0.  , 1.  , 0.  , 0.  , 1.  ]])

#### For consitency, we'll also convert the training and test target labels into Numpy arrays.

In [21]:
vs_target_train = np.array(vs_target_train)
vs_target_test = np.array(vs_target_test)

In [22]:
print (vs_target_train)
print ("\n")
print (vs_target_test)

['Yes' 'Yes' 'No' 'No' 'Yes' 'No' 'Yes' 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes' 'No'
 'Yes' 'No' 'Yes' 'No' 'Yes' 'Yes' 'No' 'No' 'Yes' 'No' 'Yes' 'No' 'Yes' 'Yes' 'No' 'No' 'Yes'
 'Yes' 'Yes' 'No' 'No' 'No' 'Yes']


['Yes' 'No' 'No' 'No' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes']


#### The following function illustrates how we can perform a k-nearest-neighbor search. The "measure" argument allows us to use either Euclidean distance or (the inverse of) Cosine similarity as the distance function:

In [23]:
def knn_search(x, D, K, measure):
    """ find K nearest neighbors of an instance x among the instances in D """
    if measure == 0:
        # euclidean distances from the other points
        dists = np.sqrt(((D - x)**2).sum(axis=1))
    elif measure == 1:
        # first find the vector norm for each instance in D as wel as the norm for vector x
        D_norm = np.array([np.linalg.norm(D[i]) for i in range(len(D))])
        x_norm = np.linalg.norm(x)
        # Compute Cosine: divide the dot product o x and each instance in D by the product of the two norms
        sims = np.dot(D,x)/(D_norm * x_norm)
        # The distance measure will be the inverse of Cosine similarity
        dists = 1 - sims
    idx = np.argsort(dists) # sorting
    # return the indexes of K nearest neighbors
    return idx[:K], dists

In [24]:
# Let's use vs_test_norm[0] as a test instance x and find its K nearest neighbors
neigh_idx, distances = knn_search(vs_test_norm[0], vs_train_norm, 5, 0)

In [25]:
vs_test.head(1)

,Income,Age,Rentals,Avg Per Visit,Gender_F,Gender_M,Genre_Action,Genre_Comedy,Genre_Drama
Cust ID,,,,,,,,,
29,15000,18,37,2.1,1,0,1,0,0


In [26]:
print (neigh_idx)
print ("\n")
vs_train.iloc[neigh_idx]

[10  7 29 35 34]




,Income,Age,Rentals,Avg Per Visit,Gender_F,Gender_M,Genre_Action,Genre_Comedy,Genre_Drama
Cust ID,,,,,,,,,
12,26000,22,32,2.9,1,0,1,0,0
18,6000,16,39,1.8,1,0,1,0,0
26,56000,35,40,2.6,1,0,1,0,0
34,29000,32,19,2.9,1,0,1,0,0
49,31000,25,42,3.4,0,1,1,0,0


In [27]:
print (distances[neigh_idx])

[0.33 0.38 0.54 0.71 1.43]


In [28]:
neigh_labels = vs_target_train[neigh_idx]
print (neigh_labels)

['Yes' 'Yes' 'Yes' 'Yes' 'Yes']


#### Now that we know the nearest neighbors, we need to find the majority class label among them. The majority class would be the class assgined to the new instance x.

In [29]:
from collections import Counter
print (Counter(neigh_labels))

Counter({'Yes': 5})


In [30]:
Counter(neigh_labels).most_common(1)

[('Yes', 5)]

#### Next, we'll use the Knn module from Chapter 2 of Machine Learning in Action. Before importing the whole module, let's illustrate what the code does by stepping through it with some specific input values.

In [31]:
dataSetSize = vs_train_norm.shape[0]
print (dataSetSize)

40


In [32]:
inX = vs_test_norm[0]   # We'll use the first instance in the test data for this example
diffMat = np.tile(inX, (dataSetSize,1)) - vs_train_norm  # Create dataSetSize copies of inX, as rows of a 2D matrix
                                                         # Compute a matrix of differences
print (diffMat[:5,:])

[[-0.3  -0.12  0.31  0.13  1.   -1.    0.    0.    0.  ]
 [-0.15 -0.12  0.47  0.21  1.   -1.    0.    0.    0.  ]
 [ 0.19  0.06  0.36  0.13  0.    0.    1.   -1.    0.  ]
 [-0.35 -0.68  0.77 -0.07  0.    0.    1.   -1.    0.  ]
 [ 0.02 -0.01  0.47  0.21  1.   -1.    0.    0.    0.  ]]


In [33]:
sqDiffMat = diffMat**2  # The matrix of squared differences
sqDistances = sqDiffMat.sum(axis=1)  # 1D array of the sum of squared differences (one element for each training instance)
distances = sqDistances**0.5  # and finally the matrix of Euclidean distances to inX
print (distances)

[1.49 1.52 1.48 1.78 1.51 1.72 2.13 0.38 1.63 2.14 0.33 2.11 1.45 2.1  1.71 1.54 1.65 1.66 1.68
 1.94 2.11 1.63 1.72 1.8  2.24 2.08 2.34 1.67 1.48 0.54 1.48 1.7  1.47 2.15 1.43 0.71 1.53 2.42
 2.11 1.46]


In [34]:
sortedDistIndicies = distances.argsort() # the indices of the training instances in increasing order of distance to inX
print (sortedDistIndicies)

[10  7 29 35 34 12 39 32 30  2 28  0  4  1 36 15  8 21 16 17 27 18 31 14 22  5  3 23 19 25 13 20 11
 38  6  9 33 24 26 37]


#### To see how the test instance should be classified, we need to find the majority class among the neighbors (here we do not use distance weighting; only a simply voting approach)

In [35]:
classCount={}
k = 5  # We'll use the top 10 neighbors
for i in range(k):
   voteIlabel = vs_target_train[sortedDistIndicies[i]]
   classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1  # add to the count of the label or retun 1 for first occurrence
   print (sortedDistIndicies[i], voteIlabel, classCount[voteIlabel])


10 Yes 1
7 Yes 2
29 Yes 3
35 Yes 4
34 Yes 5


#### Now, let's find the predicted class for the test instance.

In [36]:
import operator
# Create a dictionary for the class labels with cumulative occurrences across the neighbors as values
# Dictionary will be ordered in decreasing order of the lable values (so the majority class label will
# be the first dictonary element)
sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
print (sortedClassCount)
print (sortedClassCount[0][0])

[('Yes', 5)]
Yes


#### A better way to find the majority class given a list of class labels from neighbors:

In [37]:
from collections import Counter

k = 5  # We'll use the top 5 neighbors
vote = vs_target_train[sortedDistIndicies[0:k]]
maj_class = Counter(vote).most_common(1)

print (vote)

print (maj_class)

print ("Class label for the classified istance: ", maj_class[0][0])

['Yes' 'Yes' 'Yes' 'Yes' 'Yes']
[('Yes', 5)]
Class label for the classified istance:  Yes


#### Let's know import the whole __[kNN module from Chapter 2 of MLA book](http://facweb.cs.depaul.edu/mobasher/classes/CSC478/data/kNN.py.txt)__ and use as part of a more robust evaluation process. We will step through all test instances, use our Knn classifier to predict a class label for each instance, and in each case we compare the predicted label to the actual value from the target test labels.

In [38]:
import kNN

In [39]:
numTestVecs = len(vs_target_test)
print (numTestVecs)

10


In [40]:
errorCount = 0.0
for i in range(numTestVecs):
    # classify0 function uses Euclidean distance to find k nearest neighbors
    classifierResult = kNN.classify0(vs_test_norm[i,:], vs_train_norm, vs_target_train, 3)
    print ("the classifier came back with: %s, the real answer is: %s" % (classifierResult, vs_target_test[i]))
    if (classifierResult != vs_target_test[i]): errorCount += 1.0
        
print ("the total error rate is: %f" % (errorCount/float(numTestVecs)))

the classifier came back with: Yes, the real answer is: Yes
the classifier came back with: No, the real answer is: No
the classifier came back with: No, the real answer is: No
the classifier came back with: No, the real answer is: No
the classifier came back with: No, the real answer is: No
the classifier came back with: Yes, the real answer is: No
the classifier came back with: Yes, the real answer is: Yes
the classifier came back with: No, the real answer is: Yes
the classifier came back with: Yes, the real answer is: No
the classifier came back with: Yes, the real answer is: Yes
the total error rate is: 0.300000


#### I have added a new classifier function to the kNN module that uses Cosine similarity instead of Euclidean distance:

In [41]:
from importlib import reload
reload(kNN)

<module 'kNN' from '/Users/appobs/Desktop/hw/478/week3/kNN.py'>

In [42]:
errorCount = 0.0
for i in range(numTestVecs):
    # classify1 function uses inverse of Cosine similarity to find k nearest neighbors
    classifierResult2 = kNN.classify1(vs_test_norm[i,:], vs_train_norm, vs_target_train, 3)
    print ("the classifier came back with: %s, the real answer is: %s" % (classifierResult2, vs_target_test[i]))
    if (classifierResult2 != vs_target_test[i]): errorCount += 1.0
print ("the total error rate is: %f" % (errorCount/float(numTestVecs)))

the classifier came back with: Yes, the real answer is: Yes
the classifier came back with: No, the real answer is: No
the classifier came back with: No, the real answer is: No
the classifier came back with: No, the real answer is: No
the classifier came back with: No, the real answer is: No
the classifier came back with: Yes, the real answer is: No
the classifier came back with: Yes, the real answer is: Yes
the classifier came back with: No, the real answer is: Yes
the classifier came back with: Yes, the real answer is: No
the classifier came back with: Yes, the real answer is: Yes
the total error rate is: 0.300000
